In [8]:
import re
import json
import csv
import random
from pathlib import Path
import pandas as pd
import json
import math
from tqdm import tqdm
import openai
from typing import Dict

In [9]:
path = Path('./data/arxiv_metadata.parquet')
assert path.exists()
df = pd.read_parquet(path)
print(f"Loaded {len(df)} rows")
df.head()

Loaded 1782 rows


,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,update_date,src_pdf,text
0,2310.00014,Yong Ren,"Yong Ren, Tao Wang, Jiangyan Yi, Le Xu, Jianhu...",Fewer-token Neural Speech Codec with Time-inva...,Submitted to ICASSP 2024,None,None,None,cs.SD eess.AS,http://arxiv.org/licenses/nonexclusive-distrib...,Language model based text-to-speech (TTS) mo...,2023-10-03,2310.00014v1.pdf,FEWER-TOKEN NEURAL SPEECH CODEC WITH TIME-INVA...
1,2310.00031,Markus Marks,"Neehar Kondapaneni, Markus Marks, Manuel Knott...",Text-image Alignment for Diffusion-based Perce...,Project page: https://www.vision.caltech.edu/t...,None,None,None,cs.CV,http://arxiv.org/licenses/nonexclusive-distrib...,Diffusion models are generative models with ...,2023-10-06,2310.00031v1.pdf,Text-image Alignment for Diffusion-based Perce...
2,2310.00032,Qinghua Xu,"Qinghua Xu, Tao Yue, Shaukat Ali and Maite Arr...","Pretrain, Prompt, and Transfer: Evolving Digit...",None,None,None,None,cs.SE,http://creativecommons.org/licenses/by/4.0/,"Cyber-Physical Systems (CPSs), e.g., elevato...",2023-10-06,2310.00032v2.pdf,"PRETRAIN, PROMPT, AND TRANSFER: EVOLVING DIGIT..."
3,2310.00034,Yuzhang Shang,"Yuzhang Shang, Zhihang Yuan, Qiang Wu, Zhen Dong",PB-LLM: Partially Binarized Large Language Models,Frist work using network binarization for larg...,None,None,None,cs.LG cs.AI cs.CL,http://arxiv.org/licenses/nonexclusive-distrib...,"This paper explores network binarization, a ...",2023-10-03,2310.00034v1.pdf,PB-LLM: PARTIALLY BINARIZED LARGE LANGUAGE\nMO...
4,2310.00035,Xi Wang,"Xi Wang, Laurence Aitchison, Maja Rudolph",LoRA ensembles for large language model fine-t...,Update the title in the PDF file,None,None,None,cs.LG cs.AI,http://creativecommons.org/licenses/by/4.0/,Finetuned LLMs often exhibit poor uncertaint...,2023-10-06,2310.00035v1.pdf,Preprint. Under review\nENSEMBLE OF LOW-RANK A...


In [10]:
df_extracted = pd.read_parquet('./data/extracted_data.parquet')
print(len(df_extracted))
df_extracted.head()

582


,id,author,affiliation,longitude,latitude
0,2310.08338,Charles C. Onu,"Ubenwa Health, Montréal, Canada",-73.5673,45.5017
1,2310.08338,Charles C. Onu,"Mila-Québec AI Institute, Montréal, Canada",-73.5673,45.5017
2,2310.08338,Charles C. Onu,"McGill University, Montréal, Canada",-73.5827,45.5048
3,2310.08338,Samantha Latremouille,"Ubenwa Health, Montréal, Canada",-73.5673,45.5017
4,2310.08338,Arsenii Gorin,"Ubenwa Health, Montréal, Canada",-73.5673,45.5017


In [11]:
true_positives = []
false_positives = []
false_negatives = []

for id in df_extracted['id']:
    gold_authors = list(df[df['id'] == id]['authors'])[0]
    gold_authors = {a.strip() for a in gold_authors.split(',')}

    predicted = df_extracted[df_extracted['id'] == id]
    predicted_authors = set(predicted['author'])
    
    for author in predicted_authors:
        if author in gold_authors:
            true_positives.append((id, author))
        else:
            false_positives.append((id, author))
    
    for author in gold_authors:
        if author not in predicted_authors:
            false_negatives.append((id, author))


# round precision to 2 decimal places
precision = round(len(true_positives) / (len(true_positives) + len(false_positives)), 2)

# round recall to 2 decimal places
recall = round(len(true_positives) / (len(true_positives) + len(false_negatives)), 2)

print(f"true_positives count: {len(true_positives)}")
print(f"false_positives count: {len(false_positives)}")
print(f"false_negatives count: {len(false_negatives)}")
print("precision:", precision)
print("recall:", recall)

true_positives count: 3884
false_positives count: 364
false_negatives count: 316
precision: 0.91
recall: 0.92
